In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("HR_comma_sep.csv")
df.groupby("left").mean()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years
left,,,,,,,
0,0.666810,0.715473,3.786664,199.060203,3.380032,0.175009,0.026251
1,0.440098,0.718113,3.855503,207.419210,3.876505,0.047326,0.005321


In [3]:
df1 = df[["satisfaction_level", "average_montly_hours", "Work_accident", "promotion_last_5years"]]
Y = df.left

y = []
for i in Y:
    if i == 1:
        y.append(0)
    else:
        y.append(1)


y = pd.Series(y)
Yr = pd.concat([Y, y], axis = 1)

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(df1.values, Yr.values, test_size = 0.2)

In [42]:
inputs = tf.placeholder(tf.float32, [None, 4])
targets = tf.placeholder(tf.float32, [None, 2])

weights_1 = tf.get_variable("weights_1", [4,3])
biases_1 = tf.get_variable("biases_1", [3])

outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

weights_2 = tf.get_variable("weights_2", [3, 2])
biases_2 = tf.get_variable("biases_2", [2])

outputs = tf.matmul(outputs_1, weights_2 ) + biases_2

loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits = outputs, labels = targets)

mean_loss = tf.reduce_mean(loss)

optimize = tf.train.AdamOptimizer().minimize(mean_loss)

out_equals_targets = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))

accuracy = tf.reduce_mean(tf.cast(out_equals_targets, tf.float32))

In [43]:
sess = tf.InteractiveSession()

In [44]:
initializer  = tf.global_variables_initializer()
sess.run(initializer)

In [45]:
N, D = X_train.shape
batch_size = 1
batches_number = N // batch_size
max_epochs = 20


# In[29]:


for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0
    
    for batch_counter in range(batches_number):
        
        
        input_batch = X_train[batch_counter*batch_size: (batch_counter + 1)*batch_size]
        target_batch = Y_train[batch_counter*batch_size: (batch_counter + 1)*batch_size]
        
        _, batch_loss = sess.run([optimize, mean_loss],
                                feed_dict = {inputs : input_batch, targets : target_batch})
        
        curr_epoch_loss += batch_loss
        #print("Batch loss at batch number: %d is %.3f " %(batch_counter, batch_loss))
        
    curr_epoch_loss /= batches_number
    
    
    print('Epoch '+str(epoch_counter+1)+
          '. Mean loss: '+'{0:.3f}'.format(curr_epoch_loss))

print("End Of Training")


# In[30]:


input_batch, target_batch = X_test, Y_test

test_accuracy = sess.run([accuracy], 
    feed_dict={inputs: input_batch, targets: target_batch})

test_accuracy_percent = test_accuracy[0] * 100.

print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')


# In[ ]:


Epoch 1. Mean loss: 0.660
Epoch 2. Mean loss: 0.531
Epoch 3. Mean loss: 0.488
Epoch 4. Mean loss: 0.477
Epoch 5. Mean loss: 0.473
Epoch 6. Mean loss: 0.471
Epoch 7. Mean loss: 0.470
Epoch 8. Mean loss: 0.469
Epoch 9. Mean loss: 0.468
Epoch 10. Mean loss: 0.468
Epoch 11. Mean loss: 0.468
Epoch 12. Mean loss: 0.467
Epoch 13. Mean loss: 0.467
Epoch 14. Mean loss: 0.467
Epoch 15. Mean loss: 0.466
Epoch 16. Mean loss: 0.466
Epoch 17. Mean loss: 0.466
Epoch 18. Mean loss: 0.466
Epoch 19. Mean loss: 0.466
Epoch 20. Mean loss: 0.465
End Of Training
Test accuracy: 76.43%
